In [1]:
import socket 
import threading
import pickle
import json
import base64
import cv2
import numpy as np

HEADER = 64
PORT = 5050
SERVER = socket.gethostbyname(socket.gethostname())
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = "!DISCONNECT"

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)

def im2json(im):
    imdata = pickle.dumps(im)
    print(type(imdata))
    print(len(imdata))
    jstr = json.dumps({"image": base64.b64encode(imdata).decode('utf-8')})
    return jstr

def json2im(jstr):
    load = json.loads(jstr)
    imdata = base64.b64decode(load['image'])
    im = pickle.loads(imdata)
    return im

def handle_client(conn, addr):
    print(f"[NEW CONNECTION] {addr} connected.")

    connected = True
    while connected:
        msg_length = conn.recv(HEADER).decode(FORMAT) # first connection msg must be header
        if msg_length: #avoid msg haven't been catched or no content in it
            msg_length = int(msg_length)
            msg = conn.recv(msg_length).decode(FORMAT)# after first msg is header, others msg are messager
            if msg == DISCONNECT_MESSAGE: #close connection
                connected = False
            else:
        
                pil_img = json2im(msg)
                # pil_img.show()
                numpy_image=np.array(pil_img)
                opencv_image=cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR) 
                cv2.imshow("result",opencv_image)
            # print(f"[{addr}] {msg}")
            conn.send("Msg received".encode(FORMAT))

    conn.close()
        

def start():
    server.listen()
    print(f"[LISTENING] Server is listening on {SERVER}")
    while True: # infinite loop
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTIONS] {threading.activeCount() - 1}")


print("[STARTING] server is starting...")
start()

[STARTING] server is starting...
[LISTENING] Server is listening on 192.168.189.1
[NEW CONNECTION] ('192.168.189.1', 4260) connected.
[ACTIVE CONNECTIONS] 6


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\pytorch\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\User\anaconda3\envs\pytorch\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Local\Temp/ipykernel_24704/2403353372.py", line 45, in handle_client
    pil_img = json2im(msg)
  File "C:\Users\User\AppData\Local\Temp/ipykernel_24704/2403353372.py", line 27, in json2im
    load = json.loads(jstr)
  File "C:\Users\User\anaconda3\envs\pytorch\lib\json\__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "C:\Users\User\anaconda3\envs\pytorch\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\User\anaconda3\envs\pytorch\lib\json\decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Unterminated string startin